# Dec'24 New User Funnel - Mobile & Website - MTD

In [1]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-11-30 18:30:00' and otp_requested_at < '2024-12-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-11-30 18:30:00'  and created_at< '2024-12-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-11-30 18:30:00' and otp_verified_at <'2024-12-31 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-11-30 18:30:00' and created_at< '2024-12-31 18:30:00' 
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-11-30 18:30:00'  and created_at<'2024-12-31 18:30:00' 
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-11-30 18:30:00' and created_at<'2024-12-31 18:30:00' 
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-11-30 18:30:00' and otp_requested_at < '2024-12-31 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-11-30 18:30:00'  and created_at<'2024-12-31 18:30:00' 
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-11-30 18:30:00'   and otp_verified_at < '2024-12-31 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-11-30 18:30:00'   and created_at<'2024-12-31 18:30:00' 
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-11-30 18:30:00'  and created_at< '2024-12-31 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >'2024-11-30 18:30:00'  and created_at< '2024-12-31 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
# Updating In Marketing Analytics Sheet, C42 Tab
gsheet_name = 'Markleting Analytics' #This google sheet will be updated
tab_name = 'Registration vs install' # This particular tab is to be updated
def write_df_to_gsheet (gsheet_name,tab_name,df):
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
print("Updated_at", pd.Timestamp.now())
df

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


funnel_data downloaded sucessfully
PostgreSQL connection is closed
Updated_at 2025-01-03 14:56:48.347512


,Step,Count
0,Mobile_OTP_requested,1610
1,Mobile_OTP_Verified,1534
2,Mobile_Added_to_Cart,1427
3,Mobile_Placed_Order,320
4,Website_OTP_requested,2900
5,Website_OTP_Verified,2637
6,Website_Added_to_Cart,1955
7,Website_Placed_Order,316


# Nov'24 New User Funnel - Mobile & Website - MTD

In [1]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-10-31 18:30:00' and otp_requested_at < '2024-11-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-10-31 18:30:00'  and created_at< '2024-11-30 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-10-31 18:30:00' and otp_verified_at <'2024-11-30 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-10-31 18:30:00' and created_at< '2024-11-30 18:30:00' 
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-10-31 18:30:00'  and created_at<'2024-11-30 18:30:00' 
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-10-31 18:30:00' and created_at<'2024-11-30 18:30:00' 
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-10-31 18:30:00' and otp_requested_at < '2024-11-30 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-10-31 18:30:00'  and created_at<'2024-11-30 18:30:00' 
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-10-31 18:30:00'   and otp_verified_at < '2024-11-30 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-10-31 18:30:00'   and created_at<'2024-11-30 18:30:00' 
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-10-31 18:30:00'  and created_at< '2024-11-30 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >'2024-10-31 18:30:00'  and created_at< '2024-11-30 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
# Updating In Marketing Analytics Sheet, C42 Tab
gsheet_name = 'Markleting Analytics' #This google sheet will be updated
tab_name = 'Registration vs install' # This particular tab is to be updated
def write_df_to_gsheet (gsheet_name,tab_name,df):
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
print("Updated_at", pd.Timestamp.now())
df

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


funnel_data downloaded sucessfully
PostgreSQL connection is closed
Updated_at 2024-12-07 08:43:44.403291


,Step,Count
0,Mobile_OTP_requested,1514
1,Mobile_OTP_Verified,1437
2,Mobile_Added_to_Cart,1354
3,Mobile_Placed_Order,299
4,Website_OTP_requested,3412
5,Website_OTP_Verified,3147
6,Website_Added_to_Cart,2397
7,Website_Placed_Order,429


# Oct'24 New User Funnel - Mobile & Website - MTD

In [4]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-09-30 18:30:00' and otp_requested_at < '2024-10-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-09-30 18:30:00'  and created_at< '2024-10-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-09-30 18:30:00' and otp_verified_at <'2024-10-31 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-09-30 18:30:00' and created_at< '2024-10-31 18:30:00' 
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-09-30 18:30:00'  and created_at<'2024-10-31 18:30:00' 
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-09-30 18:30:00' and created_at<'2024-10-31 18:30:00' 
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-09-30 18:30:00' and otp_requested_at < '2024-10-31 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-09-30 18:30:00'  and created_at<'2024-10-31 18:30:00' 
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-09-30 18:30:00'   and otp_verified_at < '2024-10-31 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-09-30 18:30:00'   and created_at<'2024-10-31 18:30:00' 
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >  '2024-09-30 18:30:00'  and created_at< '2024-10-31 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at >'2024-09-30 18:30:00'  and created_at< '2024-10-31 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
# Updating In Marketing Analytics Sheet, C42 Tab
gsheet_name = 'Markleting Analytics' #This google sheet will be updated
tab_name = 'Registration vs install' # This particular tab is to be updated
def write_df_to_gsheet (gsheet_name,tab_name,df):
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
print("Updated_at", pd.Timestamp.now())
df

funnel_data downloaded sucessfully
PostgreSQL connection is closed
Updated_at 2024-11-08 18:03:03.910193


,Step,Count
0,Mobile_OTP_requested,1807
1,Mobile_OTP_Verified,1657
2,Mobile_Added_to_Cart,1575
3,Mobile_Placed_Order,365
4,Website_OTP_requested,3447
5,Website_OTP_Verified,3048
6,Website_Added_to_Cart,2241
7,Website_Placed_Order,350


# Sep'24 New User Funnel - Mobile & Website - MTD

In [1]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-08-31 18:30:00' and otp_requested_at < '2024-09-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-08-31 18:30:00'  and created_at< '2024-09-30 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-08-31 18:30:00'  and otp_verified_at < '2024-09-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-08-31 18:30:00' and created_at<'2024-09-30 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-08-31 18:30:00'  and created_at<'2024-09-30 18:30:00'
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-08-31 18:30:00' and created_at<'2024-09-30 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-08-31 18:30:00' and otp_requested_at < '2024-09-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-08-31 18:30:00' and created_at<'2024-09-30 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-08-31 18:30:00'  and otp_verified_at < '2024-09-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-08-31 18:30:00'  and created_at<'2024-09-30 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-08-31 18:30:00' and created_at<'2024-09-30 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-08-31 18:30:00'  and created_at<'2024-09-30 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
# Updating In Marketing Analytics Sheet, C42 Tab
gsheet_name = 'Markleting Analytics' #This google sheet will be updated
tab_name = 'Registration vs install' # This particular tab is to be updated
def write_df_to_gsheet (gsheet_name,tab_name,df):
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
print("Updated_at", pd.Timestamp.now())
df

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


funnel_data downloaded sucessfully
PostgreSQL connection is closed
Updated_at 2024-10-04 13:01:12.131164


,Step,Count
0,Mobile_OTP_requested,1993
1,Mobile_OTP_Verified,1867
2,Mobile_Added_to_Cart,1699
3,Mobile_Placed_Order,412
4,Website_OTP_requested,3580
5,Website_OTP_Verified,3197
6,Website_Added_to_Cart,2382
7,Website_Placed_Order,418


# Aug'24 New User Funnel - Mobile & Website - MTD

In [1]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-07-31 18:30:00' and otp_requested_at < '2024-08-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-07-31 18:30:00'  and created_at< '2024-08-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-07-31 18:30:00'  and otp_verified_at < '2024-08-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-07-31 18:30:00' and created_at<'2024-08-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-07-31 18:30:00'  and created_at<'2024-08-31 18:30:00'
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-07-31 18:30:00' and created_at<'2024-08-31 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-07-31 18:30:00' and otp_requested_at < '2024-08-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-07-31 18:30:00' and created_at<'2024-08-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-07-31 18:30:00'  and otp_verified_at < '2024-08-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-07-31 18:30:00'  and created_at<'2024-08-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-07-31 18:30:00' and created_at<'2024-08-31 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-07-31 18:30:00'  and created_at<'2024-08-31 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
# Updating In Marketing Analytics Sheet, C42 Tab
gsheet_name = 'Markleting Analytics' #This google sheet will be updated
tab_name = 'Registration vs install' # This particular tab is to be updated
def write_df_to_gsheet (gsheet_name,tab_name,df):
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
print("Updated_at", pd.Timestamp.now())
df

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


funnel_data downloaded sucessfully
PostgreSQL connection is closed
Updated_at 2024-09-13 11:43:54.984431


,Step,Count
0,Mobile_OTP_requested,1929
1,Mobile_OTP_Verified,1839
2,Mobile_Added_to_Cart,1723
3,Mobile_Placed_Order,387
4,Website_OTP_requested,3065
5,Website_OTP_Verified,2823
6,Website_Added_to_Cart,2388
7,Website_Placed_Order,418


# July'24 New User Funnel - Mobile & Website - MTD

In [1]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-06-30 18:30:00' and otp_requested_at < '2024-07-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-06-30 18:30:00'  and created_at<'2024-07-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-06-30 18:30:00'  and otp_verified_at < '2024-07-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-06-30 18:30:00' and created_at<'2024-07-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-06-30 18:30:00'  and created_at<'2024-07-31 18:30:00'
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-06-30 18:30:00' and created_at<'2024-07-31 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-06-30 18:30:00' and otp_requested_at < '2024-07-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-06-30 18:30:00' and created_at<'2024-07-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-06-30 18:30:00'  and otp_verified_at < '2024-07-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-06-30 18:30:00'  and created_at<'2024-07-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-06-30 18:30:00' and created_at<'2024-07-31 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-06-30 18:30:00'  and created_at<'2024-07-31 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
# Updating In Marketing Analytics Sheet, C42 Tab
gsheet_name = 'Markleting Analytics' #This google sheet will be updated
tab_name = 'Registration vs install' # This particular tab is to be updated
def write_df_to_gsheet (gsheet_name,tab_name,df):
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
print("Updated_at", pd.Timestamp.now())
df

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


funnel_data downloaded sucessfully
PostgreSQL connection is closed
Updated_at 2024-08-09 13:22:35.258643


,Step,Count
0,Mobile_OTP_requested,1800
1,Mobile_OTP_Verified,1720
2,Mobile_Added_to_Cart,1646
3,Mobile_Placed_Order,354
4,Website_OTP_requested,2659
5,Website_OTP_Verified,2451
6,Website_Added_to_Cart,2108
7,Website_Placed_Order,332


# June'24 New User Funnel - Mobile & Website - MTD

In [4]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-05-31 18:30:00' and otp_requested_at < '2024-06-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-05-31 18:30:00'  and created_at<'2024-06-30 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-05-31 18:30:00'  and otp_verified_at < '2024-06-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-05-31 18:30:00' and created_at<'2024-06-30 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-05-31 18:30:00'  and created_at<'2024-06-30 18:30:00'
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-05-31 18:30:00' and created_at<'2024-06-30 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-05-31 18:30:00' and otp_requested_at < '2024-06-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-05-31 18:30:00' and created_at<'2024-06-30 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-05-31 18:30:00'  and otp_verified_at < '2024-06-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-05-31 18:30:00'  and created_at<'2024-06-30 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-05-31 18:30:00' and created_at<'2024-06-30 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-05-31 18:30:00'  and created_at<'2024-06-30 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
# Updating In Marketing Analytics Sheet, C42 Tab
gsheet_name = 'Markleting Analytics' #This google sheet will be updated
tab_name = 'Registration vs install' # This particular tab is to be updated
def write_df_to_gsheet (gsheet_name,tab_name,df):
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
print("Updated_at", pd.Timestamp.now())
df

funnel_data downloaded sucessfully
PostgreSQL connection is closed
Updated_at 2024-07-13 08:57:02.829443


,Step,Count
0,Mobile_OTP_requested,1505
1,Mobile_OTP_Verified,1424
2,Mobile_Added_to_Cart,1331
3,Mobile_Placed_Order,269
4,Website_OTP_requested,2071
5,Website_OTP_Verified,1900
6,Website_Added_to_Cart,1618
7,Website_Placed_Order,238


# May'24 New User Funnel - Mobile & Website - MTD

In [1]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-04-30 18:30:00' and otp_requested_at <'2024-05-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-04-30 18:30:00' and created_at <'2024-05-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-04-30 18:30:00' and  otp_verified_at <'2024-05-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-04-30 18:30:00' and  created_at <'2024-05-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-04-30 18:30:00' and created_at <'2024-05-31 18:30:00'
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-04-30 18:30:00' and created_at <'2024-05-31 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-04-30 18:30:00' and otp_requested_at <'2024-05-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-04-30 18:30:00' and created_at <'2024-05-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-04-30 18:30:00' and  otp_verified_at <'2024-05-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-04-30 18:30:00' and created_at <'2024-05-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-04-30 18:30:00' and created_at <'2024-05-31 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-04-30 18:30:00' and created_at <'2024-05-31 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
# Updating In Marketing Analytics Sheet, C42 Tab
gsheet_name = 'Markleting Analytics' #This google sheet will be updated
tab_name = 'Registration vs install' # This particular tab is to be updated
def write_df_to_gsheet (gsheet_name,tab_name,df):
    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
    worksheet = sh.worksheet(tab_name)
    set_with_dataframe(worksheet,df)
write_df_to_gsheet(gsheet_name,tab_name,df)
print("Updated_at", pd.Timestamp.now())
df

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


funnel_data downloaded sucessfully
PostgreSQL connection is closed
Updated_at 2024-06-07 12:07:51.292686


,Step,Count
0,Mobile_OTP_requested,1908
1,Mobile_OTP_Verified,1784
2,Mobile_Added_to_Cart,1704
3,Mobile_Placed_Order,386
4,Website_OTP_requested,2621
5,Website_OTP_Verified,2376
6,Website_Added_to_Cart,2010
7,Website_Placed_Order,337


# April'24 New User Funnel - Mobile & Website - MTD

In [ ]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-03-31 18:30:00' and otp_requested_at < '2024-04-30 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-03-31 18:30:00' and created_at < '2024-04-30 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-03-31 18:30:00' and otp_verified_at < '2024-04-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-03-31 18:30:00' and  created_at < '2024-04-30 18:30:00' 
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-03-31 18:30:00' and created_at < '2024-04-30 18:30:00'
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-03-31 18:30:00' and created_at < '2024-04-30 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-03-31 18:30:00' and otp_requested_at < '2024-04-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-03-31 18:30:00' and created_at < '2024-04-30 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-03-31 18:30:00' and otp_verified_at <  '2024-04-30 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-03-31 18:30:00' and created_at < '2024-04-30 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-03-31 18:30:00' and created_at < '2024-04-30 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-03-31 18:30:00' and created_at < '2024-04-30 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
## Updating In Marketing Analytics Sheet, C42 Tab
#gsheet_name = 'Markleting Analytics' #This google sheet will be updated
#tab_name = 'Registration vs install' # This particular tab is to be updated
#def write_df_to_gsheet (gsheet_name,tab_name,df):
#    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
#    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
#    worksheet = sh.worksheet(tab_name)
#    set_with_dataframe(worksheet,df)
#write_df_to_gsheet(gsheet_name,tab_name,df)
#print("Updated_at", pd.Timestamp.now())
df

# March'24 New User Funnel - Mobile & Website - MTD

In [ ]:
import csv
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-02-29 18:30:00' and  otp_requested_at < '2024-03-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-02-29 18:30:00' and created_at < '2024-03-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-02-29 18:30:00' and otp_verified_at < '2024-03-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-02-29 18:30:00' and created_at < '2024-03-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-02-29 18:30:00' and created_at < '2024-03-31 18:30:00'
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-02-29 18:30:00' and created_at < '2024-03-31 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-02-29 18:30:00' and otp_requested_at < '2024-03-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-02-29 18:30:00' and created_at < '2024-03-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-02-29 18:30:00' and otp_verified_at < '2024-03-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-02-29 18:30:00' and created_at < '2024-03-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-02-29 18:30:00' and created_at < '2024-03-31 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-02-29 18:30:00' and created_at < '2024-03-31 18:30:00'
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()

        ## Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

        
## Updating In Marketing Analytics Sheet, C42 Tab
#gsheet_name = 'Markleting Analytics' #This google sheet will be updated
#tab_name = 'Registration vs install' # This particular tab is to be updated
#def write_df_to_gsheet (gsheet_name,tab_name,df):
#    gc = gspread.service_account(filename=r"C:\Users\kumar.rohit.KUMARROHIT-LPT\Desktop\summary-automation-project-fd46b6ab2eba.json")
#    sh = gc.open_by_key("1XPW1oNGxZIS0Qigi3FA9mT5M9eH7ievgEwK2qoyVV1I") #Key Of the google sheet - Summary_Epharmacy
#    worksheet = sh.worksheet(tab_name)
#    set_with_dataframe(worksheet,df)
#write_df_to_gsheet(gsheet_name,tab_name,df)
#print("Updated_at", pd.Timestamp.now())
df

# February'24 New User Funnel - Mobile & Website

In [ ]:
#customer Journey Funnels
try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-01-31 18:30:00' and otp_requested_at < '2024-02-29 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-01-31 18:30:00' and created_at < '2024-02-29 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-01-31 18:30:00' and otp_verified_at < '2024-02-29 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-01-31 18:30:00' and created_at < '2024-02-29 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-01-31 18:30:00' and created_at < '2024-02-29 18:30:00' 
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-01-31 18:30:00' and created_at < '2024-02-29 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2024-01-31 18:30:00' and otp_requested_at < '2024-02-29 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-01-31 18:30:00' and created_at < '2024-02-29 18:30:00' 
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2024-01-31 18:30:00' and otp_verified_at < '2024-02-29 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-01-31 18:30:00' and created_at < '2024-02-29 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-01-31 18:30:00' and created_at < '2024-02-29 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2024-01-31 18:30:00' and created_at < '2024-02-29 18:30:00' 
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        #print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()
        
        # Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        #print("PostgreSQL connection is closed")
print("February Funnel")
df

# January'24 New User Funnel - Mobile & Website

In [ ]:
#customer Journey Funnels
try:
    # Create an SSH tunnel to the Remote Server(EC2 Instance)
    with SSHTunnelForwarder(('65.1.183.184', 22),
        ssh_username='ubuntu',
        ssh_pkey="C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem",
        remote_bind_address=('emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com', 5432),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database='efrprod',
            user='emamireaduser',
            password='emamireadaccess'
        )
        cursor = connection.cursor()
        
        # Execute queries for mobile and website data
        queries = [
            # Mobile queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2023-12-31 18:30:00' and otp_requested_at < '2024-01-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2023-12-31 18:30:00' and created_at < '2024-01-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at >'2023-12-31 18:30:00' and otp_verified_at < '2024-01-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2023-12-31 18:30:00' and created_at < '2024-01-31 18:30:00'
                AND registration_source = 1
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2023-12-31 18:30:00' and created_at < '2024-01-31 18:30:00'
                AND registration_source = 1
            ))and order_status_logs.channel = 1 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2023-12-31 18:30:00' and created_at < '2024-01-31 18:30:00'
                AND registration_source = 1
            ))and channel = 1 and to_state = 3
            ''',

            # Website queries
            '''
            SELECT count(distinct user_id) as new_users 
            FROM PHONES 
            WHERE otp_requested_at > '2023-12-31 18:30:00' and otp_requested_at < '2024-01-31 18:30:00' 
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2023-12-31 18:30:00' and created_at < '2024-01-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT count(distinct user_id) as registered 
            FROM PHONES 
            WHERE otp_verified_at > '2023-12-31 18:30:00' and otp_verified_at < '2024-01-31 18:30:00'
            AND user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2023-12-31 18:30:00' and created_at < '2024-01-31 18:30:00'
                AND registration_source = 4
            );
            ''',
            '''
            SELECT 
            count(distinct modified_by_id) as add_to_cart 
            FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2023-12-31 18:30:00' and created_at < '2024-01-31 18:30:00'
                AND registration_source = 4
            ))and order_status_logs.channel = 4 and to_state = 0 and  from_state = 0
            ''',
            '''
            SELECT count(distinct modified_by_id) as placed_order
             FROM order_status_logs where order_id in  
            (
            SELECT id
            FROM orders 
            WHERE user_id IN (
                SELECT id 
                FROM users 
                WHERE created_at > '2023-12-31 18:30:00' and created_at < '2024-01-31 18:30:00' 
                AND registration_source = 4
            ))and channel = 4 and to_state = 3
            '''
        ]

        funnel_data = []

        for query in queries:
            cursor.execute(query)
            record = cursor.fetchone()
            funnel_data.append(record[0])

        # Aggregate the results into a dictionary named data
        data = {
            'Step': ['Mobile_OTP_requested', 'Mobile_OTP_Verified', 'Mobile_Added_to_Cart', 'Mobile_Placed_Order',
                     'Website_OTP_requested', 'Website_OTP_Verified', 'Website_Added_to_Cart', 'Website_Placed_Order'],
            'Count': funnel_data
        }
        
        # Write aggregated data to a CSV file(Funnel_data)
        #with open('funnel_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        #    writer = csv.writer(csvfile)
        #    writer.writerow(['Step', 'Count'])
        #    for row in data.items():
        #        writer.writerow(row)
        
        # Create a funnel chart
        df = pd.DataFrame(data)
        df.to_csv("funnel_data.csv") #Saving funnel data as Data Frame
        #print("funnel_data downloaded sucessfully")
        #plt.figure(figsize=(12, 8))
        #bars = plt.barh(df['Step'], df['Count'], color='skyblue')
        #plt.xlabel('Count')
        #plt.ylabel('Step')
        #plt.title('User Journey Funnel - MTD')
        #plt.gca().invert_yaxis()
        
        # Add counts to each bar
        #for bar in bars:
        #    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{int(bar.get_width())}',
        #             va='center', ha='left')
        #plt.show()

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        #print("PostgreSQL connection is closed")
print("January Funnels")
df